### `Body2Hands` Data Preparation

In [ ]:
# import numpy as np
# from bvh.skeleton import process_bvhfile, process_bvhkeyframe

# body_idx = [17,18,19,45,46,47]
# hand_idx = [20,21,22,25,26,27,30,31,32,35,36,37,40,41,48,49,50,53,54,55,58,59,60,63,64,65,68,69]

# body_data_b2h = []
# hand_data_b2h = []
# check = 0

# for data_type in ["b","c"]:
#     if data_type == "b":
#         data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
#                         "11_01","12_02","13_01","14_01","15_02","16_02","17_01","18_01","19_01","20_01",
#                         "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]
#     elif data_type == "c":
#         data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
#                         "11_01","12_02","13_01","14_01","15_02","17_01","18_01","19_01","20_01",
#                         "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]
        
#     for data_number in data_numbers:
#         mocap = process_bvhfile("../data/bvh/VAAI_DIRECT_" + data_number + '_' + data_type + "_M1.bvh", DEBUG=False)
#         for i in range(mocap.frames):
#             new_frame = process_bvhkeyframe(mocap.keyframes[i], mocap.root, mocap.dt * i)

#         _, rot = mocap.get_frames_rotations()
#         rot = np.array(rot)
#         rot = rot[:, 1:].reshape(rot.shape[0], -1, 3)[:, :, [2, 0, 1]]
        
#         L = rot.shape[0]
#         rot_body = rot[:, body_idx, :]
#         rot_hand = rot[:, hand_idx, :]

#         rot_body = rot_body.reshape(L, -1)
#         rot_hand = rot_hand.reshape(L, -1)

#         for i in range(int(L/32)-1):
#             patch = rot_body[32*i:32*(i+2):]
#             body_data_b2h.append(patch)

#         for i in range(int(L/32)-1):
#             patch = rot_hand[32*i:32*(i+2):]
#             hand_data_b2h.append(patch)

#         body_data_b2h.append(rot_body[-64:,:])
#         hand_data_b2h.append(rot_hand[-64:,:])

# body_data_b2h = np.array(body_data_b2h)
# np.save('../data/b2h_train/nc_train_body.npy', body_data_b2h)
# print("train body data shape: ", body_data_b2h.shape)

# hand_data_b2h = np.array(hand_data_b2h)
# np.save('../data/b2h_train/nc_train_hands.npy', hand_data_b2h)
# print("train hands data shape: ", hand_data_b2h.shape)

# data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
#                 "11_01","12_02","13_01","14_01","15_02","16_02","17_01","18_01","19_01","20_01",
#                 "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]

# for data_number in data_numbers:
#     mocap = process_bvhfile("../data/bvh/VAAI_DIRECT_" + data_number + "_a_M1.bvh", DEBUG=False)
#     for i in range(mocap.frames):
#         new_frame = process_bvhkeyframe(mocap.keyframes[i], mocap.root, mocap.dt * i)

#     _, rot = mocap.get_frames_rotations()
#     rot = np.array(rot)
#     rot = rot[:, 1:].reshape(rot.shape[0], -1, 3)[:, :, [2, 0, 1]]
        
#     L = rot.shape[0]
#     rot_body = rot[:, body_idx, :]
#     rot_hand = rot[:, hand_idx, :]
    
#     body_data = np.zeros((1,L,18))
#     hand_data = np.zeros((1,L,84))

#     body_data[0,:,:] = rot_body.reshape(L, -1)
#     hand_data[0,:,:] = rot_hand.reshape(L, -1)

#     np.save('../data/b2h_test/full_bodies'+ data_number[:2] +'.npy', body_data)
#     print("body shape: ", body_data.shape)

#     np.save('../data/b2h_test/full_hands'+ data_number[:2] +'.npy', hand_data)
#     print("hands shape: ", hand_data.shape)

### `Body2Hands` Results Visualization

In [1]:
import copy,cv2,glob,os,pickle
import numpy as np
import mediapy as media
import networkx as nx
import matplotlib.pyplot as plt
from util import rpy2r,create_folder_if_not_exists, rpy2R
from mujoco_parser import MuJoCoParserClass
from kin_chain import KinematicChainClass
np.set_printoptions(precision=2)
%matplotlib inline
%config InlineBackend.figure_format='retina'
print ("networkx version:[%s]"%(nx.__version__))

Ready.
networkx version:[3.1]


### Load bvh & Initialize kinematic chain

In [ ]:
from bvh.skeleton import process_bvhfile, process_bvhkeyframe
from collections import deque

data_number = "15_02"
mocap = process_bvhfile("../data/bvh/VAAI_DIRECT_" + data_number + "_a_M1.bvh", DEBUG=False)

body_idx = [17,18,19,45,46,47]
hand_idx = [20,21,22,25,26,27,30,31,32,35,36,37,40,41,48,49,50,53,54,55,58,59,60,63,64,65,68,69]
hand_tip_idx = [24,29,34,39,43,52,57,62,67,71]
hand_idx_names = ['thumb_03_rEnd','index_03_rEnd','middle_03_rEnd','ring_03_rEnd','pinky_03_rEnd',
                  'thumb_03_lEnd','index_03_lEnd','middle_03_lEnd','ring_03_lEnd','pinky_03_lEnd']

node = mocap.root
deq = deque()
joint_names = []
with open('../data/offsets.pkl', 'rb') as f:
    offsets = pickle.load(f)

for i in range(mocap.frames):
    new_frame = process_bvhkeyframe(mocap.keyframes[i], mocap.root, mocap.dt * i)

pos_header, pos = mocap.get_frames_worldpos()
rot_header, rot = mocap.get_frames_rotations()

rot = np.array(rot)
rot = rot[:, 1:].reshape(rot.shape[0], -1, 3)[:, :, [2, 0, 1]]

pos = np.array(pos)/10
pos = pos[:, 1:].reshape(pos.shape[0], -1, 3)[:, :, [2, 0, 1]]

kin_chain = KinematicChainClass(name='NC chain')
kin_chain.add_joint(name='World',p=np.array([0,0,0]),R=rpy2r(np.radians([0,0,0])))

kin_chain.add_joint(name=node.name,parent_name='World',
        p_offset=np.array([0,0,1.8]),R_offset=rpy2r(np.radians([0,0,0])))
joint_names.append(node.name)
while len(node.children) > 0 or len(deq) > 0:
    if len(deq) > 0:
        node = deq.pop()
        kin_chain.add_joint(name=node.name,parent_name=node.parent.name,
                p_offset=offsets[node.name],R_offset=rpy2r(np.radians([0,0,0]))
                )
        joint_names.append(node.name)

    for child in node.children:
        deq.append(child)

### Visualize original data

In [ ]:
import os
import cv2

CHECK_LOOP = False
SAVE_VIDEO = False

video = []
tick = 0

L = pos.shape[0]
p_joints = np.empty_like(pos)
R_offset_joints = np.empty((pos.shape[0], pos.shape[1], 3, 3))
pos_hand_tip = np.zeros_like(pos[:, hand_tip_idx,:])

# Initialize MuJoCo viewer
xml_path = '../asset/object/floor_sky.xml'
env = MuJoCoParserClass(name='Floor',rel_xml_path=xml_path,VERBOSE=False)
env.init_viewer(viewer_title='Kinematic Chain',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=177,distance=25.0,elevation=-30,lookat=[0.0,0.1,8],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,
                  jointrgba=[0.2,0.6,0.8,0.6])
env.reset()

while tick < L:
    # Step
    env.step()
    i = tick % rot.shape[0]

    # Forward Kinematics
    R_offsets = []
    kin_chain.chain.nodes[1]['p_offset'] = pos[i, 0, :]
    
    for j in range(rot.shape[1]):
        R_offsets.append(rpy2R(np.radians(rot[i,j]), [0,2,1]))
        R_offset_joints[i-1, j, :, :] = rpy2R(np.radians(rot[i,j]), [0,2,1])

    kin_chain.set_R_offset(joint_names, R_offsets)
    kin_chain.forward_kinematics()
    for j, joint_name in enumerate(joint_names):
        kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p'] =  kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p'][[2,1,0]]
        kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p'][0] *= -1
        p_joints[i-1, j, :] = kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p']

    # Render
    if env.loop_every(tick_every=1):
        env.plot_T(p=np.zeros(3,),R=np.eye(3),
                   PLOT_AXIS=True,axis_len=0.1,axis_width=0.01,PLOT_SPHERE=False,
                   label='World (tick:[%d] time:[%.1f]s)'%(tick,env.get_sim_time()))
        # Plot chain in MuJoCo
        kin_chain.plot_chain_mujoco(env,r_link=0.1)

        for (frame_number, joint_name) in enumerate(hand_idx_names):
            pos_hand_tip[tick, frame_number, :] = kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p']/10
        
        env.render()
        frame = env.grab_image()
        video.append(frame)
        
    tick = tick + 1
    if tick == L:
        if CHECK_LOOP: tick = 0

env.close_viewer()

# Plot the final scene
plt.figure(figsize=(6,4)); plt.imshow(frame)
plt.title("Tick:[%d] Time:[%.2f]sec"%(env.tick,env.get_sim_time()),fontsize=9)
plt.show()

if SAVE_VIDEO:
    shape = video[0].shape
    video_folder = ("../video/")
    if not os.path.isdir(video_folder): os.mkdir(video_folder)

    video_path = os.path.join(video_folder, data_number[:2] + "baseline.mp4")
    video_fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
    fps = 50
    video_out = cv2.VideoWriter(video_path, video_fourcc, fps, (shape[1], shape[0]), True)

    for i in range(len(video)):
        video_frame = cv2.cvtColor(video[i], cv2.COLOR_BGR2RGB)
        video_out.write(video_frame)
    video_out.release()

### Generated data

In [ ]:
generated_hands = np.load("../data/b2h_test/results/predicted_hands%.2d.npy" % (int(data_number[:2])))
generated_bodies = np.load("../data/b2h_test/full_bodies%.2d.npy" % (int(data_number[:2])))

generated_bodies = generated_bodies.reshape(generated_bodies.shape[1], -1, 3)
generated_hands = generated_hands.reshape(generated_hands.shape[1], -1, 3)

if generated_hands.shape[0]!=generated_bodies.shape[0]:
    temp = np.zeros((generated_bodies.shape[0], generated_hands.shape[1], generated_hands.shape[2]))
    temp[:generated_hands.shape[0], :, :] = generated_hands
    temp[-1,:,:] = generated_hands[-1,:,:]
    generated_hands = temp

data = rot.copy()
data[:, hand_idx, :] = generated_hands

rot = np.array(data)

### Visualize generated data

In [ ]:
import os
import cv2

CHECK_LOOP = False
SAVE_VIDEO = False

video = []
tick = 0

L = pos.shape[0]
p_joints = np.empty_like(pos)
R_offset_joints = np.empty((pos.shape[0], pos.shape[1], 3, 3))
generated_pos_hand_tip = np.empty_like(pos_hand_tip)

# Initialize MuJoCo viewer
xml_path = '../asset/object/floor_sky.xml'
env = MuJoCoParserClass(name='Floor',rel_xml_path=xml_path,VERBOSE=False)
env.init_viewer(viewer_title='Kinematic Chain',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=177,distance=25.0,elevation=-30,lookat=[0.0,0.1,8],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,
                  jointrgba=[0.2,0.6,0.8,0.6])
env.reset()

while tick < L:
    # Step
    env.step()
    i = tick % rot.shape[0]

    # Forward Kinematics
    R_offsets = []
    kin_chain.chain.nodes[1]['p_offset'] = pos[i, 0, :]
    
    for j in range(rot.shape[1]):
        R_offsets.append(rpy2R(np.radians(rot[i,j]), [0,2,1]))
        R_offset_joints[i-1, j, :, :] = rpy2R(np.radians(rot[i,j]), [0,2,1])

    kin_chain.set_R_offset(joint_names, R_offsets)
    kin_chain.forward_kinematics()
    for j, joint_name in enumerate(joint_names):
        kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p'] =  kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p'][[2,1,0]]
        kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p'][0] *= -1
        p_joints[i-1, j, :] = kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p']

    # Render
    if env.loop_every(tick_every=1):
        env.plot_T(p=np.zeros(3,),R=np.eye(3),
                   PLOT_AXIS=True,axis_len=0.1,axis_width=0.01,PLOT_SPHERE=False,
                   label='World (tick:[%d] time:[%.1f]s)'%(tick,env.get_sim_time()))
        # Plot chain in MuJoCo
        kin_chain.plot_chain_mujoco(env,r_link=0.1)
        for (frame_number, joint_name) in enumerate(hand_idx_names):
            generated_pos_hand_tip[tick, frame_number, :] = kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p']/10
        
        env.render()
        frame = env.grab_image()
        video.append(frame)
        
    tick = tick + 1
    if tick == L:
        if CHECK_LOOP: tick = 0

env.close_viewer()

# Plot the final scene
plt.figure(figsize=(6,4)); plt.imshow(frame)
plt.title("Tick:[%d] Time:[%.2f]sec"%(env.tick,env.get_sim_time()),fontsize=9)
plt.show()

if SAVE_VIDEO:
    shape = video[0].shape
    video_folder = ("../video/")
    if not os.path.isdir(video_folder): os.mkdir(video_folder)

    video_path = os.path.join(video_folder, data_number[:2] + "baseline.mp4")
    video_fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
    fps = 50
    video_out = cv2.VideoWriter(video_path, video_fourcc, fps, (shape[1], shape[0]), True)

    for i in range(len(video)):
        video_frame = cv2.cvtColor(video[i], cv2.COLOR_BGR2RGB)
        video_out.write(video_frame)
    video_out.release()

In [ ]:
a = pos_hand_tip; b = generated_pos_hand_tip
diff = a - b
dist = np.sqrt(diff[:,:,0]**2 + diff[:,:,1]**2 + diff[:,:,2]**2)
print(np.average(sum(dist)/L))

print(dist.shape)

### For all sequences

In [2]:
import pickle
import numpy as np
from util import rpy2r,rpy2R
from mujoco_parser import MuJoCoParserClass
from kin_chain import KinematicChainClass
from bvh.skeleton import process_bvhfile, process_bvhkeyframe
from collections import deque

data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
                "11_01","12_02","13_01","14_01","15_02","16_02","17_01","18_01","19_01","20_01",
                "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]

body_idx = [17,18,19,45,46,47]
hand_idx = [20,21,22,25,26,27,30,31,32,35,36,37,40,41,48,49,50,53,54,55,58,59,60,63,64,65,68,69]
hand_tip_idx = [24,29,34,39,43,52,57,62,67,71]
hand_idx_names = ['thumb_03_rEnd','index_03_rEnd','middle_03_rEnd','ring_03_rEnd','pinky_03_rEnd',
                  'thumb_03_lEnd','index_03_lEnd','middle_03_lEnd','ring_03_lEnd','pinky_03_lEnd']

dist_list = []
l2_list = []

for data_number in data_numbers:
    mocap = process_bvhfile("../data/bvh/VAAI_DIRECT_" + data_number + "_a_M1.bvh", DEBUG=False)

    node = mocap.root
    deq = deque()
    joint_names = []
    with open('../data/offsets.pkl', 'rb') as f:
        offsets = pickle.load(f)

    for i in range(mocap.frames):
        new_frame = process_bvhkeyframe(mocap.keyframes[i], mocap.root, mocap.dt * i)

    pos_header, pos = mocap.get_frames_worldpos()
    rot_header, rot = mocap.get_frames_rotations()

    rot = np.array(rot)
    rot = rot[:, 1:].reshape(rot.shape[0], -1, 3)[:, :, [2, 0, 1]]

    pos = np.array(pos)/10
    pos = pos[:, 1:].reshape(pos.shape[0], -1, 3)[:, :, [2, 0, 1]]

    kin_chain = KinematicChainClass(name='NC chain')
    kin_chain.add_joint(name='World',p=np.array([0,0,0]),R=rpy2r(np.radians([0,0,0])))

    kin_chain.add_joint(name=node.name,parent_name='World',
            p_offset=np.array([0,0,1.8]),R_offset=rpy2r(np.radians([0,0,0])))
    joint_names.append(node.name)
    while len(node.children) > 0 or len(deq) > 0:
        if len(deq) > 0:
            node = deq.pop()
            kin_chain.add_joint(name=node.name,parent_name=node.parent.name,
                    p_offset=offsets[node.name],R_offset=rpy2r(np.radians([0,0,0]))
                    )
            joint_names.append(node.name)

        for child in node.children:
            deq.append(child)

    tick = 0

    L = pos.shape[0]
    p_joints = np.empty_like(pos)
    R_offset_joints = np.empty((pos.shape[0], pos.shape[1], 3, 3))
    pos_hand_tip = np.zeros_like(pos[:, hand_tip_idx,:])

    # Initialize MuJoCo viewer
    xml_path = '../asset/object/floor_sky.xml'
    env = MuJoCoParserClass(name='Floor',rel_xml_path=xml_path,VERBOSE=False)
    env.init_viewer(viewer_title='Kinematic Chain',viewer_width=1200,viewer_height=800,
                    viewer_hide_menus=True)
    env.update_viewer(azimuth=177,distance=25.0,elevation=-30,lookat=[0.0,0.1,8],
                    VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                    contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                    VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,
                    jointrgba=[0.2,0.6,0.8,0.6])
    env.reset()

    while tick < L:
        # Step
        env.step()
        i = tick % rot.shape[0]

        # Forward Kinematics
        R_offsets = []
        kin_chain.chain.nodes[1]['p_offset'] = pos[i, 0, :]
        
        for j in range(rot.shape[1]):
            R_offsets.append(rpy2R(np.radians(rot[i,j]), [0,2,1]))
            R_offset_joints[i-1, j, :, :] = rpy2R(np.radians(rot[i,j]), [0,2,1])

        kin_chain.set_R_offset(joint_names, R_offsets)
        kin_chain.forward_kinematics()
        for j, joint_name in enumerate(joint_names):
            kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p'] =  kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p'][[2,1,0]]
            kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p'][0] *= -1
            p_joints[i-1, j, :] = kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p']

        # Render
        if env.loop_every(tick_every=1):
            env.plot_T(p=np.zeros(3,),R=np.eye(3),
                    PLOT_AXIS=True,axis_len=0.1,axis_width=0.01,PLOT_SPHERE=False,
                    label='World (tick:[%d] time:[%.1f]s)'%(tick,env.get_sim_time()))
            # Plot chain in MuJoCo
            kin_chain.plot_chain_mujoco(env,r_link=0.1)

            for (frame_number, joint_name) in enumerate(hand_idx_names):
                pos_hand_tip[tick, frame_number, :] = kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p']/10
            
        tick = tick + 1

    env.close_viewer()

    generated_hands = np.load("../data/b2h_test/results/predicted_hands%.2d.npy" % (int(data_number[:2])))
    generated_bodies = np.load("../data/b2h_test/full_bodies%.2d.npy" % (int(data_number[:2])))

    generated_bodies = generated_bodies.reshape(generated_bodies.shape[1], -1, 3)
    generated_hands = generated_hands.reshape(generated_hands.shape[1], -1, 3)

    if generated_hands.shape[0]!=generated_bodies.shape[0]:
        temp = np.zeros((generated_bodies.shape[0], generated_hands.shape[1], generated_hands.shape[2]))
        temp[:generated_hands.shape[0], :, :] = generated_hands
        temp[-1,:,:] = generated_hands[-1,:,:]
        generated_hands = temp

    data = rot.copy()
    data[:, hand_idx, :] = generated_hands

    rot = np.array(data)

    tick = 0

    L = pos.shape[0]
    p_joints = np.empty_like(pos)
    R_offset_joints = np.empty((pos.shape[0], pos.shape[1], 3, 3))
    generated_pos_hand_tip = np.empty_like(pos_hand_tip)

    # Initialize MuJoCo viewer
    xml_path = '../asset/object/floor_sky.xml'
    env = MuJoCoParserClass(name='Floor',rel_xml_path=xml_path,VERBOSE=False)
    env.init_viewer(viewer_title='Kinematic Chain',viewer_width=1200,viewer_height=800,
                    viewer_hide_menus=True)
    env.update_viewer(azimuth=177,distance=25.0,elevation=-30,lookat=[0.0,0.1,8],
                    VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                    contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                    VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,
                    jointrgba=[0.2,0.6,0.8,0.6])
    env.reset()

    while tick < L:
        # Step
        env.step()
        i = tick % rot.shape[0]

        # Forward Kinematics
        R_offsets = []
        kin_chain.chain.nodes[1]['p_offset'] = pos[i, 0, :]
        
        for j in range(rot.shape[1]):
            R_offsets.append(rpy2R(np.radians(rot[i,j]), [0,2,1]))
            R_offset_joints[i-1, j, :, :] = rpy2R(np.radians(rot[i,j]), [0,2,1])

        kin_chain.set_R_offset(joint_names, R_offsets)
        kin_chain.forward_kinematics()
        for j, joint_name in enumerate(joint_names):
            kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p'] =  kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p'][[2,1,0]]
            kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p'][0] *= -1
            p_joints[i-1, j, :] = kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p']

        # Render
        if env.loop_every(tick_every=1):
            env.plot_T(p=np.zeros(3,),R=np.eye(3),
                    PLOT_AXIS=True,axis_len=0.1,axis_width=0.01,PLOT_SPHERE=False,
                    label='World (tick:[%d] time:[%.1f]s)'%(tick,env.get_sim_time()))
            # Plot chain in MuJoCo
            kin_chain.plot_chain_mujoco(env)
            for (frame_number, joint_name) in enumerate(hand_idx_names):
                generated_pos_hand_tip[tick, frame_number, :] = kin_chain.chain.nodes[kin_chain.get_joint_idx(joint_name)]['p']/10
            
        tick = tick + 1

    env.close_viewer()

    a = pos_hand_tip; b = generated_pos_hand_tip
    diff = a - b
    dist = np.sqrt(diff[:,:,0]**2 + diff[:,:,1]**2 + diff[:,:,2]**2)

    dist_list.append(dist)
    print("%s L2 Distance (mm): %.4f" % (data_number[:2], np.average(sum(dist)/L)*1000))
    l2_list.append(np.average(sum(dist)/L)*1000)


01 L2 Distance (mm): 14.9570
02 L2 Distance (mm): 16.2883
03 L2 Distance (mm): 13.6441
04 L2 Distance (mm): 10.3398
05 L2 Distance (mm): 6.4828
06 L2 Distance (mm): 11.4263
07 L2 Distance (mm): 11.1247
08 L2 Distance (mm): 6.2368
09 L2 Distance (mm): 21.9621
10 L2 Distance (mm): 10.1398
11 L2 Distance (mm): 19.4767
12 L2 Distance (mm): 27.1229
13 L2 Distance (mm): 11.8723
14 L2 Distance (mm): 12.8930
15 L2 Distance (mm): 26.1382
16 L2 Distance (mm): 23.8192
17 L2 Distance (mm): 17.3229
18 L2 Distance (mm): 20.4308
19 L2 Distance (mm): 21.7735
20 L2 Distance (mm): 12.7223
21 L2 Distance (mm): 12.4972
22 L2 Distance (mm): 10.9837
23 L2 Distance (mm): 26.4774
24 L2 Distance (mm): 25.8110
25 L2 Distance (mm): 19.8131
26 L2 Distance (mm): 10.8868
27 L2 Distance (mm): 13.4120


In [3]:
l2_list = np.array(l2_list)
print("L2 Distance: %.2f +/- %.2f" % (np.mean(l2_list), np.std(l2_list)))

L2 Distance: 16.15 +/- 6.15


L2 Distance: 16.08 +/- 9.73